In [27]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import seaborn as sns
from statsmodels import api as sm
from matplotlib import pyplot as plt
from pandas.plotting import register_matplotlib_converters
import os
import numpy as np
import math
import validate

In [28]:
data_dir='fixed_csv_label_ver2'
original = pd.read_csv( os.path.join(data_dir, 'learning_data_10.csv') )
df = pd.read_csv( os.path.join(data_dir, 'learning_data_10.csv') )
df=df[df['day']<=9]
df

,original,label,time,day
0,53.4,0,15,0
1,52.8,0,15,0
2,52.8,0,15,0
3,53.2,0,15,0
4,53.6,0,15,0
...,...,...,...,...
3995345,45.9,0,5,9
3995346,45.6,0,5,9
3995347,45.8,0,5,9
3995348,45.7,0,5,9


In [3]:
# trend
period=3
for i in set(df['day']):
    df.loc[df['day']==i, 'trend']=df.loc[df['day']==i, 'original'].rolling(period+1).mean()
df=df.dropna().reset_index(drop=True)
df

,original,label,time,day,trend
0,53.2,0,15,0,53.050
1,53.6,0,15,0,53.100
2,54.0,0,15,0,53.400
3,54.7,0,15,0,53.875
4,55.7,0,15,0,54.500
...,...,...,...,...,...
3995315,45.9,0,5,9,45.700
3995316,45.6,0,5,9,45.625
3995317,45.8,0,5,9,45.725
3995318,45.7,0,5,9,45.750


In [6]:
#差分値
for i in set(df['day']):
    df.loc[df['day']==i, 'diff']=df.loc[df['day']==i, 'original'].diff(1)
df=df.dropna().reset_index(drop=True)
df

,original,label,time,day,diff
0,52.8,0,15,0,-0.6
1,52.8,0,15,0,0.0
2,53.2,0,15,0,0.4
3,53.6,0,15,0,0.4
4,54.0,0,15,0,0.4
...,...,...,...,...,...
3995335,45.9,0,5,9,0.3
3995336,45.6,0,5,9,-0.3
3995337,45.8,0,5,9,0.2
3995338,45.7,0,5,9,-0.1


In [29]:
#変化率
for i in set(df['day']):
    df.loc[df['day']==i, 'pct_c']=df.loc[df['day']==i, 'original'].pct_change(1)
df=df.dropna().reset_index(drop=True)
df

,original,label,time,day,pct_c
0,52.8,0,15,0,-0.011236
1,52.8,0,15,0,0.000000
2,53.2,0,15,0,0.007576
3,53.6,0,15,0,0.007519
4,54.0,0,15,0,0.007463
...,...,...,...,...,...
3995335,45.9,0,5,9,0.006579
3995336,45.6,0,5,9,-0.006536
3995337,45.8,0,5,9,0.004386
3995338,45.7,0,5,9,-0.002183


In [6]:
for i in set(df['day']):
    df.loc[df['day']==i, 'pct']=df.loc[df['day']==i, 'original']/df.loc[df['day']==i, 'original'].shift(1)
df=df.dropna().reset_index(drop=True)
df

,original,label,time,day,trend,diff,pct_c,pct
0,54.7,0,15,0,53.875,0.7,0.012963,1.012963
1,55.7,0,15,0,54.500,1.0,0.018282,1.018282
2,55.5,0,15,0,54.975,-0.2,-0.003591,0.996409
3,55.7,0,15,0,55.400,0.2,0.003604,1.003604
4,55.6,0,15,0,55.625,-0.1,-0.001795,0.998205
...,...,...,...,...,...,...,...,...
1151432,53.6,0,6,2,54.550,-0.7,-0.012891,0.987109
1151433,53.8,0,6,2,54.200,0.2,0.003731,1.003731
1151434,53.1,0,6,2,53.700,-0.7,-0.013011,0.986989
1151435,53.0,0,6,2,53.375,-0.1,-0.001883,0.998117


In [31]:
def make_data(x, str,w):
    x=pd.DataFrame(x)
    window=w
    for i in range(1, window):
        column=f'{str}_lag{i}'
        x[column]=x[str].shift(i)
    return x

def make_label(x, w):
    window=w
    x['label_all']=x['label']
    for i in range(1, window):
        x.loc[(x['label_all']==1)|(x['label'].shift(i)==1), 'label_all']=1
        x.loc[(x['label_all']==0)&(x['label'].shift(i)==0), 'label_all']=0
    
    x['label']=x['label_all']
    return x[['label']]

df_list=[]
for i in set(df['day']):
    lag=[] 
    window=5
    lag.append(make_data(df.loc[df['day']==i, ['pct_c']], 'pct_c', window))
    lag.append(df.loc[df['day']==i, ['day']])
    lag.append(make_label(df.loc[df['day']==i, ['label']], window))
    df_add_lag=pd.concat(lag, axis=1)
    df_list.append(df_add_lag)
df=pd.concat(df_list)
df=df.dropna().reset_index(drop=True)
df

,pct_c,pct_c_lag1,pct_c_lag2,pct_c_lag3,pct_c_lag4,day,label
0,0.007463,0.007519,0.007576,0.000000,-0.011236,0,0
1,0.012963,0.007463,0.007519,0.007576,0.000000,0,0
2,0.018282,0.012963,0.007463,0.007519,0.007576,0,0
3,-0.003591,0.018282,0.012963,0.007463,0.007519,0,0
4,0.003604,-0.003591,0.018282,0.012963,0.007463,0,0
...,...,...,...,...,...,...,...
3995295,0.006579,0.004405,-0.010893,-0.002174,0.006565,9,0
3995296,-0.006536,0.006579,0.004405,-0.010893,-0.002174,9,0
3995297,0.004386,-0.006536,0.006579,0.004405,-0.010893,9,0
3995298,-0.002183,0.004386,-0.006536,0.006579,0.004405,9,0


In [9]:
df['original_st']=df['original']*100/df['original']
df

,original,original_lag1,original_lag2,original_lag3,original_lag4,original_lag5,original_lag6,original_lag7,original_lag8,original_lag9,day,label,original_st
0,54.9,54.9,55.0,55.7,55.8,55.6,55.7,55.5,55.7,54.7,0,0,100.0
1,54.9,54.9,54.9,55.0,55.7,55.8,55.6,55.7,55.5,55.7,0,0,100.0
2,55.0,54.9,54.9,54.9,55.0,55.7,55.8,55.6,55.7,55.5,0,0,100.0
3,55.0,55.0,54.9,54.9,54.9,55.0,55.7,55.8,55.6,55.7,0,0,100.0
4,55.2,55.0,55.0,54.9,54.9,54.9,55.0,55.7,55.8,55.6,0,0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1151423,53.6,54.3,55.1,55.2,55.6,56.2,56.4,56.8,56.4,56.0,2,0,100.0
1151424,53.8,53.6,54.3,55.1,55.2,55.6,56.2,56.4,56.8,56.4,2,0,100.0
1151425,53.1,53.8,53.6,54.3,55.1,55.2,55.6,56.2,56.4,56.8,2,0,100.0
1151426,53.0,53.1,53.8,53.6,54.3,55.1,55.2,55.6,56.2,56.4,2,0,100.0


In [12]:
for i in range(1, 10):
    cl_b=f'original_lag{i}'
    cl_a=f'st_lag{i}'
    df[cl_a]=df[cl_b]*100/df['original']
df[df.columns[12:]]

,original_st,st_lag1,st_lag2,st_lag3,st_lag4,st_lag5,st_lag6,st_lag7,st_lag8,st_lag9
0,100.0,100.000000,100.182149,101.457195,101.639344,101.275046,101.457195,101.092896,101.457195,99.635701
1,100.0,100.000000,100.000000,100.182149,101.457195,101.639344,101.275046,101.457195,101.092896,101.457195
2,100.0,99.818182,99.818182,99.818182,100.000000,101.272727,101.454545,101.090909,101.272727,100.909091
3,100.0,100.000000,99.818182,99.818182,99.818182,100.000000,101.272727,101.454545,101.090909,101.272727
4,100.0,99.637681,99.637681,99.456522,99.456522,99.456522,99.637681,100.905797,101.086957,100.724638
...,...,...,...,...,...,...,...,...,...,...
1151423,100.0,101.305970,102.798507,102.985075,103.731343,104.850746,105.223881,105.970149,105.223881,104.477612
1151424,100.0,99.628253,100.929368,102.416357,102.602230,103.345725,104.460967,104.832714,105.576208,104.832714
1151425,100.0,101.318267,100.941620,102.259887,103.766478,103.954802,104.708098,105.838041,106.214689,106.967985
1151426,100.0,100.188679,101.509434,101.132075,102.452830,103.962264,104.150943,104.905660,106.037736,106.415094


In [36]:
test_day=[0]
train=df[(df['label']==0)&(~df['day'].isin(test_day))][df.columns[:-2]].reset_index(drop=True)
test=df[(df['day'].isin(test_day))][df.columns[:-2]].reset_index(drop=True)
test

,pct_c,pct_c_lag1,pct_c_lag2,pct_c_lag3,pct_c_lag4
0,0.007463,0.007519,0.007576,0.000000,-0.011236
1,0.012963,0.007463,0.007519,0.007576,0.000000
2,0.018282,0.012963,0.007463,0.007519,0.007576
3,-0.003591,0.018282,0.012963,0.007463,0.007519
4,0.003604,-0.003591,0.018282,0.012963,0.007463
...,...,...,...,...,...
431975,0.010508,0.001754,0.021505,-0.003571,0.010830
431976,0.006932,0.010508,0.001754,0.021505,-0.003571
431977,0.001721,0.006932,0.010508,0.001754,0.021505
431978,0.010309,0.001721,0.006932,0.010508,0.001754


In [37]:
nk = 10
neigh = NearestNeighbors(n_neighbors=nk)
neigh.fit(train)
d = neigh.kneighbors(test)[0]

d = np.mean(d, axis=1)
mx = np.max(d)
d = d / mx

In [38]:
# 評価用のデータを改めて用意し、異常度を割り当てる
test_v=original[original['day'].isin(test_day)][['original', 'label']].reset_index(drop=True)
test_v['d']=d[0]
test_v.loc[window:, 'd']=d
d=test_v['d'].values

In [39]:
thr_df=pd.read_csv('day_thr.csv')
thr=float(thr_df.loc[thr_df['day'].isin(test_day), 'thr'])
precision, recall=validate.validate(test_v, d, thr)
print(f'適合率：{precision}')
print(f'再現率：{recall}')

適合率：0.8606403013182674
再現率：0.7606837606837606
